In [1]:
from utilities.utils import get_fulldata, process_land_area

usecols = ["name_of_ryot", "caste", "6", "7", "8"]
df = get_fulldata(**{"usecols": usecols, "low_memory": False}).pipe(process_land_area)
print(len(df))
df.head()

/home/lsys/land/venv_land/lib/python3.10/site-packages/pandas_flavor/register.py:161: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


38589908


,name_of_ryot,caste,acres,decimals,hectare,tt_area_acre
0,कपिलदेव पाठक,ब्रहामण,0.0,66.0,0.0,0.66
1,कपिलदेव पाठक,ब्रहामण,0.0,8.0,0.0,0.08
2,कपिलदेव पाठक,ब्रहामण,0.0,6.0,0.0,0.06
3,कपिलदेव पाठक,ब्रहामण,0.0,3.0,0.0,0.03
4,कपिलदेव पाठक,ब्रहामण,0.0,36.0,0.0,0.36


In [2]:
df_ryot_caste = (
    df.dropna(subset="name_of_ryot")
    .select_columns(["name_of_ryot", "caste"])
    .drop_duplicates(subset=["name_of_ryot"])
    .reset_index(drop=True)
)
# df_ryot_caste.to_csv("../data/ryot_hindi_caste.csv.gz", index=False, compression="gzip")
print(len(df_ryot_caste))
df_ryot_caste

3197303


,name_of_ryot,caste
0,कपिलदेव पाठक,ब्रहामण
1,किशोरी राय,यादव
2,किताब महतो,कुर्मी
3,जगदेव राय,यादव
4,अकलु राय,यादव
...,...,...
3197298,चरीत्तर चौधरी,कलवार
3197299,विहार ग्राम उधोग संघ सरैया,NaN
3197300,बिहार सरकार कृषी विभाग वीजगुणा,NaN
3197301,मु. मंचनिया,कोइरी


In [3]:
visible_pattern = r'[°\.\[\]\(\)\{\}\\\/;:&"\'\+\-\|०-९⁪a-zA-Z0-9]'
unicode_pattern = r'[\u0000-\u001F\u007F-\u009F\u200B-\u200F\u202A-\u202E\u206A-\u206F]'
combined_pattern = f'{visible_pattern}|{unicode_pattern}'

matches = df_ryot_caste[df_ryot_caste["name_of_ryot"].str.contains(combined_pattern, na=False, regex=True)]
print(f"Found {len(matches)} rows with problematic characters")
print(matches["name_of_ryot"].sample(n=min(100, len(matches))).tolist())

Found 732395 rows with problematic characters
['मो0 इसरार खां', 'पप्\u200dपू कूमार सिह', 'स्\u200dव0 महादेव चौहान', 'नविन्द्र प्र. सिंह', 'मु0 सुवचनी', 'विश्\u200dवनाथ व एतवा टुडू', 'श्\u200dयामदेव  मंडल', 'श्री हरि ना.झा', 'भोला प्र0 गुप्ता', 'मृत्\u200dयून्\u200dजय कुमार', 'मो0 इन्दाज', 'जमुना प्र.दास', 'कुलवुल दुसाध वगै0', 'भीम सेन प्र0 यादव', 'राचन्\u200dद्र प्रसाद मिश्र', 'हरगोविन्\u200dद महतो', 'विजेन्\u200dद्र नारायण चौधरी', 'शे0 मनीर वो शे0 याकुब', 'मो.अनवर खान', 'अब्दुल खालीक ई.', 'मु.सोमरतीया', 'वाकु मरन्\u200dडी,फ्रांसिस मरन्\u200dडी', 'चोखा कुम्\u200dहार', 'मु. वाएज', 'शे० बुझवान शे० दुखी', 'श्री कमला कान्\u200dत मंडल', 'मु. मुनार चौधराइ', 'मु.जमीर', 'म0 हवीब', 'इन्\u200dद्रदेव पाडे वो', 'मो.बीबी मुन्ना खातून', 'श्री मो.रैणु देवी', 'विनय कु0 झा', 'मसो० कमला देवी ]', 'देवना0 मेंहता', 'मु. योशोदा', 'क़ष्\u200dणमणी देवी', 'खेदारू यादव वो श्\u200dयामलाल यादव', 'सिताराम बुप्\u200dता', 'मू0 मसीर वो जकीर वो मो सगीर', 'बिहार सरकार ब्\u200dलौक कपोड', 'बिनोद कु0 विशवास', 'मो.मजवुन नी

In [4]:
def clean_hindi_names(df, name_column='name_of_ryot'):
    r"""
    Aggressively clean Hindi names - keep ONLY valid Devanagari and spaces.
    
    Removes ALL:
    - Punctuation and special characters
    - English letters and numbers
    - Devanagari digits
    - Unicode control characters
    - Basically everything except Devanagari letters and spaces
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    name_column : str
        Name of the column containing names to clean
        
    Returns:
    --------
    pd.DataFrame
        Dataframe with cleaned names (empty names removed)
    """
    # Basic punctuation and brackets
    punctuation = r'[°\.\,\;\:\!\?\[\]\(\)\{\}]'
    # Special symbols
    symbols = r'[&"\'\`\~\@\#\$\%\^\*\_\=\+\-\|\\\/\<\>]'
    # English letters and numbers
    alphanumeric = r'[a-zA-Z0-9]'
    # Devanagari digits
    devanagari_digits = r'[०-९]'
    # Unicode control/formatting characters
    unicode_control = r'[\u0000-\u001F\u007F-\u009F\u200B-\u200F\u202A-\u202E\u206A-\u206F]'

    combined_pattern = f'{punctuation}|{symbols}|{alphanumeric}|{devanagari_digits}|{unicode_control}'
    df[name_column] = df[name_column].str.replace(combined_pattern, '', regex=True)
    
    df[name_column] = df[name_column].str.replace(r'\s+', ' ', regex=True)
    df[name_column] = df[name_column].str.strip()
    
    return df

In [5]:
df_ryot_caste = clean_hindi_names(df_ryot_caste)
# df_ryot_caste.to_csv("../data/ryot_hindi_caste.csv.gz", index=False, compression="gzip")
df_ryot_caste

,name_of_ryot,caste
0,कपिलदेव पाठक,ब्रहामण
1,किशोरी राय,यादव
2,किताब महतो,कुर्मी
3,जगदेव राय,यादव
4,अकलु राय,यादव
...,...,...
3197298,चरीत्तर चौधरी,कलवार
3197299,विहार ग्राम उधोग संघ सरैया,NaN
3197300,बिहार सरकार कृषी विभाग वीजगुणा,NaN
3197301,मु मंचनिया,कोइरी


In [6]:
matches = df_ryot_caste[df_ryot_caste["name_of_ryot"].str.contains(combined_pattern, na=False, regex=True)]
print(f"Found {len(matches)} rows with problematic characters")
try:
    print(matches["name_of_ryot"].sample(n=100).tolist())
except ValueError:
    pass

Found 0 rows with problematic characters


In [7]:
df_ryot_caste[df_ryot_caste['name_of_ryot'].str.len() == 0]

,name_of_ryot,caste
63,,यादव
1480,,हजाम
3098,,NaN
3170,,वीरसुल प्रसाद
3521,,कोईरी
...,...,...
3189110,,यादव
3189215,,NaN
3189336,,NaN
3189535,,कुर्मी


In [8]:
df_ryot_caste = df_ryot_caste[df_ryot_caste['name_of_ryot'].str.len() > 0]
df_ryot_caste.to_csv("../data/ryot_hindi_caste.csv.gz", index=False, compression="gzip")
df_ryot_caste

,name_of_ryot,caste
0,कपिलदेव पाठक,ब्रहामण
1,किशोरी राय,यादव
2,किताब महतो,कुर्मी
3,जगदेव राय,यादव
4,अकलु राय,यादव
...,...,...
3197298,चरीत्तर चौधरी,कलवार
3197299,विहार ग्राम उधोग संघ सरैया,NaN
3197300,बिहार सरकार कृषी विभाग वीजगुणा,NaN
3197301,मु मंचनिया,कोइरी
